In [1]:
import numpy as npa
import pandas as pd

import pytz
import locale
from datetime import datetime

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('ThExam').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 11:54:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 11:54:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
local_timezone = pytz.timezone('Asia/Bangkok')
execute_time = datetime.now(local_timezone)
year = execute_time.year
month = execute_time.month
day = execute_time.day

print(execute_time)
print("year:{} | month={} | day={}".format(year,month,day))

2024-03-03 11:54:08.747054+07:00
year:2024 | month=3 | day=3


## Load to Postgres 

In [25]:
schema = StructType([
    StructField("user",StringType(),True),
    StructField("timestamp",TimestampType(),True),
    StructField("hours",DecimalType(5,2),True),
    StructField("project",StringType(),True),
  ])

In [26]:
TABLE = 'dailycheckins'

In [27]:
path = "../tm-staging-zone/table={}/year={}/month={}/day={}/".format(TABLE,year,month,day)
df = spark.read.schema(schema).csv(path, header=True)

In [28]:
df.describe().toPandas()

,summary,user,hours,project
0,count,20495,20500,20500
1,mean,None,2.071275,None
2,stddev,None,2.0047747481961506,None
3,min,arya,0.00,airflow
4,max,ygritte,30.00,workshops


In [29]:
df = df.withColumnRenamed('user', 'username').drop('user')

In [31]:
df.show()

+--------+--------------------+-----+--------------------+
|username|           timestamp|hours|             project|
+--------+--------------------+-----+--------------------+
|     ned| 2019-09-27 07:00:00| 8.00|              bizdev|
|     ned| 2019-09-26 07:00:00| 1.00|cultureandmanagement|
|     ned| 2019-09-26 07:00:00| 1.50|          project-00|
|     ned| 2019-09-26 07:00:00| 1.00|          project-43|
|   jaime| 2018-12-21 07:00:00| 0.50|          project-47|
|   jaime| 2018-12-21 07:00:00| 3.50|          project-47|
|   jaime| 2018-12-20 07:00:00| 1.50|          project-00|
|   jaime| 2018-12-20 07:00:00| 1.50|          project-30|
|   jaime| 2018-12-20 07:00:00| 1.00|          project-47|
|   jaime| 2018-12-20 07:00:00| 3.00|          project-47|
| catelyn|2018-11-26 21:47:...| 0.60|         opsandadmin|
| catelyn|2018-11-26 21:47:...| 0.45|         opsandadmin|
| catelyn|2018-11-26 21:47:...| 0.40|cultureandmanagement|
| catelyn|2018-11-26 21:47:...| 0.08|            securit

24/03/03 12:00:25 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///Users/prachy/Git/tm-pipeline/tm-staging-zone/table=dailycheckins/year=2024/month=3/day=3/part-00000-7bd6a002-cf4e-4993-a925-cfd19d405ce9-c000.csv


In [11]:
# aurora_postgres_url = "jdbc:postgresql://database-1.cluster-cqnnuqtyqggk.ap-southeast-1.rds.amazonaws.com:5432/tm"
# aurora_postgres_properties = {
#     "user": "prachya",
#     "password": "prachya2024",
#     "driver": "org.postgresql.Driver"
# }

In [13]:
# df.limit(10).write\
# .format("jdbc")\
# .option("url", aurora_postgres_url)\
# .option("dbtable", "dailycheckin")\
# .option("user", aurora_postgres_properties["user"])\
# .option("password", aurora_postgres_properties["password"])\
# .option("driver", aurora_postgres_properties["driver"])\
# .mode("append")\
# .save()